In [125]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.keras import models, layers, utils 
from sklearn import metrics, preprocessing
import os
import gc
import sys

import math
import re
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate
sns.set_style("darkgrid")

gc.collect()


24

## recommender system 
the two main functions are 
get_similar_movies(movie_title, min_ratings_count = 0)

recommend_movies(user_id , n_movies= 10)


In [2]:
path = "D:\\netflix dataset\\netflix-prize-data"


In [4]:
start = datetime.now()
if not os.path.isfile('data.csv'):
    data = open('data.csv', mode='w')
    row = list()
    files = os.listdir(path)
    print(files)
    for file in files:
        if file.startswith('combined_data'):
        # if file.startswith('combined_data_1'):   # use this line to process only one file
            print("Reading ratings from {}\n".format(file))
            with open(os.path.join(path,file)) as f:
                for line in f:
                    line = line.strip()
                    if line.endswith(':'):
                        # All below are ratings for this movie, until another movie appears.
                        movie_id = line.replace(':', '')
                    else:
                        row = [x for x in line.split(',')]
                        row.insert(0, movie_id)
                        data.write(','.join(row))
                        data.write('\n')
data.close()
print('Time taken :', datetime.now() - start)


['combined_data_1.txt', 'combined_data_2.txt', 'combined_data_3.txt', 'combined_data_4.txt', 'movie_titles.csv', 'probe.txt', 'qualifying.txt', 'README']
Reading ratings from combined_data_1.txt

Time taken : 0:01:16.384216


In [3]:
df = pd.read_csv('data.csv', names=['Movie_Id', 'Cust_Id', 'Rating', 'Date'])


In [4]:
df.head()


,Movie_Id,Cust_Id,Rating,Date
0,1,1488844,3,2005-09-06
1,1,822109,5,2005-05-13
2,1,885013,4,2005-10-19
3,1,30878,4,2005-12-26
4,1,823519,3,2004-05-03


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100480507 entries, 0 to 100480506
Data columns (total 4 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   Movie_Id  int64 
 1   Cust_Id   int64 
 2   Rating    int64 
 3   Date      object
dtypes: int64(3), object(1)
memory usage: 3.0+ GB


In [6]:
df['Movie_Id'] = df['Movie_Id'].astype(np.uint32)
df['Cust_Id'] = df['Cust_Id'].astype(np.uint32)
df['Rating'] = df['Rating'].astype(np.uint8)
df.drop(['Date'], axis=1, inplace=True)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100480507 entries, 0 to 100480506
Data columns (total 3 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   Movie_Id  uint32
 1   Cust_Id   uint32
 2   Rating    uint8 
dtypes: uint32(2), uint8(1)
memory usage: 862.4 MB


In [64]:
f = ['count', 'mean']

df_movie_summary = df.groupby('Movie_Id')['Rating'].agg(f)
df_movie_summary.index = df_movie_summary.index.map(int)
movie_benchmark = round(df_movie_summary['count'].quantile(0.7), 0)
drop_movie_list = df_movie_summary[df_movie_summary['count']
                                   < movie_benchmark].index

print('Movie minimum times of review: {}'.format(movie_benchmark))

df_cust_summary = df.groupby('Cust_Id')['Rating'].agg(f)
df_cust_summary.index = df_cust_summary.index.map(int)
cust_benchmark = round(df_cust_summary['count'].quantile(0.7), 0)
drop_cust_list = df_cust_summary[df_cust_summary['count']
                                 < cust_benchmark].index

print('Customer minimum times of review: {}'.format(cust_benchmark))

print('Original Shape: {}'.format(df.shape))
df = df[~df['Movie_Id'].isin(drop_movie_list)]
df = df[~df['Cust_Id'].isin(drop_cust_list)]
print('After Trim Shape: {}'.format(df.shape))


print("number of movies deleted is " , len(drop_movie_list))
print("number of customers deleted is " , len(drop_cust_list))

Movie minimum times of review: 12331.0
Customer minimum times of review: 550.0
Original Shape: (71833509, 3)
After Trim Shape: (27358794, 3)
number of movies deleted is  3732
number of customers deleted is  101012


In [8]:
# del df_cust_summary, df_movie_summary, drop_movie_list, drop_cust_list
gc.collect()
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 71833509 entries, 693 to 100479585
Data columns (total 3 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   Movie_Id  uint32
 1   Cust_Id   uint32
 2   Rating    uint8 
dtypes: uint32(2), uint8(1)
memory usage: 1.1 GB


In [108]:
df_title = pd.read_csv("D:\\netflix dataset\\netflix-prize-data\\movie_titles.csv", encoding="ISO-8859-1",
                        header=None, names=['Movie_Id', 'Year', 'Name'], usecols=[0, 1, 2])
df_title.set_index('Movie_Id', inplace=True)
df_title['Year'].fillna(0, inplace=True)
df_title['Year'] = df_title['Year'].astype(int)
df_title.head(10)


,Year,Name
Movie_Id,,
1,2003,Dinosaur Planet
2,2004,Isle of Man TT 2004 Review
3,1997,Character
4,1994,Paula Abdul's Get Up & Dance
5,2004,The Rise and Fall of ECW
6,1997,Sick
7,1992,8 Man
8,2004,What the #$*! Do We Know!?
9,1991,Class of Nuke 'Em High 2


In [10]:
df_pivot = pd.pivot_table(df, values='Rating',
                      index='Cust_Id', columns='Movie_Id')

print(df_pivot.shape)


(144380, 5332)


In [12]:
df_pivot.info()
# nearly 6 gb of ram is used by pivot table

<class 'pandas.core.frame.DataFrame'>
UInt64Index: 144380 entries, 6 to 2649429
Columns: 5332 entries, 3 to 17769
dtypes: float64(5332)
memory usage: 5.7 GB


In [139]:
# min count is min count of ratings given by a customer
def get_similar_movies(movie_title, min_ratings_count=0):
    print("For movie ({})".format(movie_title))
    print("- Top 10 movies recommended based on Pearsons'R correlation - ")
    i = int(df_title.index[df_title['Name'] == movie_title][0])
    target = df_pivot[i]
    similar_to_target = df_pivot.corrwith(target)
    corr_target = pd.DataFrame(similar_to_target, columns=['PearsonR'])
    corr_target.dropna(inplace = True)
    corr_target = corr_target.sort_values('PearsonR', ascending = False)
    corr_target.index = corr_target.index.map(int)
    corr_target = corr_target.join(df_title).join(df_movie_summary)[['PearsonR', 'Name', 'count', 'mean']]
    corr_target.rename(columns={"Name": "movie name", "count": "number of ratings"  , "mean": "average rating" ,
                                "PearsonR": "Pearson correlation coefficient"}, inplace=True)
    
    print(corr_target[corr_target['number of ratings']>min_ratings_count][1:11].to_string(index=False))

In [140]:
get_similar_movies("Lord of the Rings: The Fellowship of the Ring", 30000)
# you can see it outputs similar movies 


For movie (Lord of the Rings: The Fellowship of the Ring)
- Top 10 movies recommended based on Pearsons'R correlation - 


NameError: name 'df_pivot' is not defined

In [53]:
get_similar_movies("Fight Club")
get_similar_movies("Fight Club", 80000)
# here it actually ouputs movies of the same genre or the the same director
# and min count produces better recommendations if used correctly

For movie (Fight Club)
- Top 10 movies recommended based on Pearsons'R correlation - 
 Pearson correlation coefficient                            movie name  number of ratings  average rating
                        0.380089                            12 Monkeys              76475        3.776084
                        0.378151                                Snatch              68818        3.848005
                        0.364983                               Memento             127318        3.851427
                        0.357912                          Pulp Fiction             152078        4.094235
                        0.347596                         Trainspotting              24446        3.885953
                        0.343847 Eternal Sunshine of the Spotless Mind             110159        3.695304
                        0.333834                        Reservoir Dogs              90450        4.003847
                        0.326851  The Royal Tenenbaums: Bonus Mate

In [54]:
get_similar_movies("Saving Private Ryan", 0)
# here it ouputs similar war movies


For movie (Saving Private Ryan)
- Top 10 movies recommended based on Pearsons'R correlation - 
 Pearson correlation coefficient        movie name  number of ratings  average rating
                        0.348649   Black Hawk Down              77597        3.859608
                        0.343554        Braveheart             135601        4.294423
                        0.328930         Apollo 13              98116        4.082576
                        0.321378         Gladiator             150592        4.202713
                        0.321019      Forrest Gump             181508        4.299910
                        0.315738  Band of Brothers              37514        4.513328
                        0.315564           Platoon              64175        3.986038
                        0.303892  We Were Soldiers              64917        4.005299
                        0.295405  Schindler's List             101141        4.458004
                        0.292609 Sands of Iwo

In [60]:
# del df_pivot
gc.collect()

75859

### collaborative filtering
Singular Value Decomposition (SVD)

In [136]:
# first we can observe like movies of a customer
def get_prev_liked_movies(user_id, rating=5, num_movies=10):
    liked_movies = df[(df['Cust_Id'] == user_id) & (df['Rating'] == rating)]
    liked_movies.set_index('Movie_Id', inplace=True)
    liked_movies = liked_movies.join(df_title)[['Name', 'Year', 'Rating']]
    print("Liked movies by user {} are : ".format(user_id))
    # print(liked_movies.unique())
    print(liked_movies.head(num_movies).to_string(index=False))


get_prev_liked_movies(1987434)


Liked movies by user 1987434 are : 
                                                  Name  Year  Rating
                                      X2: X-Men United  2003       5
                                               Ed Wood  1994       5
   Monty Python's The Meaning of Life: Special Edition  1983       5
                                            Roger & Me  1989       5
                                                  Jaws  1975       5
                                The Simpsons: Season 1  1989       5
                                       Rosemary's Baby  1968       5
                 Eternal Sunshine of the Spotless Mind  2004       5
Pirates of the Caribbean: The Curse of the Black Pearl  2003       5
                                  Being John Malkovich  1999       5


In [124]:
reader = Reader()

# get just top 100K rows for faster run time
data = Dataset.load_from_df(df[['Cust_Id', 'Movie_Id', 'Rating']][:1000000], reader)
#data.split(n_folds=3)
trainset = data.build_full_trainset()

algo = SVD()
cross_validate(algo, data, measures=['RMSE'] , cv=5, verbose=10)


Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9089  0.9081  0.9112  0.9098  0.9112  0.9098  0.0012  
Fit time          13.61   12.53   15.94   15.05   12.56   13.94   1.36    
Test time         4.96    3.90    5.16    5.17    4.27    4.69    0.52    


{'test_rmse': array([0.90888754, 0.90808392, 0.91117295, 0.90983842, 0.91122518]),
 'fit_time': (13.61171841621399,
  12.529053211212158,
  15.939460277557373,
  15.049062490463257,
  12.560122966766357),
 'test_time': (4.958907842636108,
  3.896749258041382,
  5.16167950630188,
  5.1707069873809814,
  4.274164438247681)}

In [137]:
print(sys.getsizeof(df) /1_000_000 , "MB")


465.099514 MB


In [120]:



# getting full dataset

trainset = data.build_full_trainset()
algo.fit(trainset)


In [188]:
def recommend_movies(user_id, min_ratings_count=0, n_movies=10):
    user = df_title.copy()
    user = user.reset_index()
    # user = user[~user['Movie_Id'].isin(drop_movie_list)]
    user['Estimated rating'] = user['Movie_Id'].apply(
        lambda x: algo.predict(user_id, x).est)
    # print(user.head())
    user.set_index('Movie_Id', inplace=True)
    
    # print(user.head())
    user = user.join(df_movie_summary)[['Name' , 'Estimated rating', 'count']]
    user = user[user['count'] > min_ratings_count]
    user = user.sort_values('Estimated rating', ascending=False)
    user.rename(columns={"Name": "movie name", "count": "number of ratings" } , inplace=True)
    print(user.head(n_movies).to_string(index=False))



In [189]:
recommend_movies(23443, 50000, 10)


            movie name  Estimated rating  number of ratings
      X2: X-Men United          3.954219            73490.0
        Reservoir Dogs          3.919622            66263.0
     Kill Bill: Vol. 2          3.805133            76858.0
       American Beauty          3.754870           102478.0
 Bridget Jones's Diary          3.720629            59528.0
Something's Gotta Give          3.672999            76604.0
   Lost in Translation          3.588129            85498.0
    Behind Enemy Lines          3.588129            57317.0
               Twister          3.588129            90249.0
       Lethal Weapon 2          3.588129            55018.0


In [190]:
algo.predict(324, 2).est


3.58812875